In [2]:
import os
import sys

In [7]:
sys.path.append(os.path.join(os.path.abspath('../..'), 'src'))

In [8]:
from bs4 import BeautifulSoup as bs

In [52]:
import mysql_utils
import mongo_utils

In [9]:
with mongo_utils.collWithBP() as coll:
    pdf = mongo_utils.dfDocsFromCursor(coll.find(filter = {'html' : {'$ne' : ''}}, limit=25))

In [10]:
pdf['soup'] = pdf.html.apply(lambda x: bs(x, 'html.parser'))

In [56]:
soup = pdf['soup'][0]

In [57]:
tables = soup.find_all('table')

In [58]:
len(tables)

18

In [61]:
t1 = \
soup.find('table', {'style' : "margin-right:0px; margin-top: 10px; margin-bottom: 10px; width: 100%;",
                    'align' : "center"}
         )
t1

<table align="center" cellpadding="0" cellspacing="0" style="margin-right:0px; margin-top: 10px; margin-bottom: 10px; width: 100%;">
<tr>
<td style="vertical-align: top; width: 35%; border: 1px solid #999;padding:0px;">
<table cellspacing="0" style="width: 100%; margin: 0px; padding: 0px;">
<tr><td colspan="2" style="background-color: #ced0d2; padding: 0px; text-align: center; font-weight: bold;border-bottom: 1px solid #555;border-top: 1px solid #555;"><a href="/gascripts/billsummaries/billsummaries.pl?Session=2017&amp;BillID=H1">View Available Bill Summaries</a> </td></tr>
<tr class="sectionBG1">
<th class="tableHeader" style="border-bottom: 1px solid #555; padding-left: 5px;">Bill Text</th><th class="tableHeader" nowrap="" style="border-bottom: 1px solid #555;">Fiscal Note</th>
</tr>
<tr>
<td nowrap="" style="border-bottom: 1px dotted #DCDCDC; padding-left:4px; padding-bottom:4px; vertical-align: top;"><a href="/Sessions/2017/Bills/House/PDF/H1v0.pdf">Filed</a> <span style="font: 9px

## Metadata Stuffs

In [73]:
la_row = soup.find('td', string='Last Action:')

In [78]:
la_row.find_next('td', {'class' : 'tableText'})

<td class="tableText" style="padding-left: 10px; padding-bottom: 10px; width: 85%; vertical-align: top;">Adopted on 01/11/2017</td>

In [83]:
tds_2_find = ['Last Action:', 'Attributes:', ]
ths_2_find = ['Sponsors:', 'Counties:', 'Statutes:', 'Keywords:']

In [84]:
tds = soup.find_all('td')
target_tds = [td for td in tds if td.text in tds_2_find]

In [86]:
ths = soup.find_all('th')
target_ths = [th for th in ths if th.text in ths_2_find]

In [87]:
len(target_ths)

4

In [100]:
meta = []
for tag in target_tds:
    label = tag.text.strip(':')
    content = tag.find_next('td', {'class' : 'tableText'}).text
    meta.append({'label' : label,
                 'content' : content})
for tag in target_ths:
    label = tag.text.strip(':')
    content = tag.find_next('td', {'class' : 'tableText'})
    if label == 'Sponsors':
        values = content.find_all('a')
        sponsors = []
        for v in values:
            sponsors.append({'Name' : v.text,
                             'Link' : v['href']})
        content = sponsors
    else:
        content = content.text
    meta.append({'label' : label,
                 'content' : content})

In [103]:
meta

[{'content': 'Adopted on 01/11/2017', 'label': 'Last Action'},
 {'content': 'Public;       ', 'label': 'Attributes'},
 {'content': [{'Link': '/gascripts/members/viewMember.pl?sChamber=H&nUserID=389',
    'Name': 'Lewis'}],
  'label': 'Sponsors'},
 {'content': 'No counties specifically cited', 'label': 'Counties'},
 {'content': 'No affected General Statutes\n', 'label': 'Statutes'},
 {'content': 'ADOPTED, GENERAL ASSEMBLY, RESOLUTIONS, SIMPLE, RULES, HOUSE & SENATE',
  'label': 'Keywords'}]

In [97]:
for m in meta:
    print(m[0].text.strip(':'))
    print('\t' + m[1].text)

Last Action
	Adopted on 01/11/2017
Attributes
	Public;       
Sponsors
	Lewis;   (Primary) 
Counties
	No counties specifically cited
Statutes
	No affected General Statutes

Keywords
	ADOPTED, GENERAL ASSEMBLY, RESOLUTIONS, SIMPLE, RULES, HOUSE & SENATE


In [107]:
def get_meta(soup):
    
    tds_2_find = ['Last Action:', 'Attributes:', ]
    ths_2_find = ['Sponsors:', 'Counties:', 'Statutes:', 'Keywords:']

    tds = soup.find_all('td')
    target_tds = [td for td in tds if td.text in tds_2_find]
    
    ths = soup.find_all('th')
    target_ths = [th for th in ths if th.text in ths_2_find]
    
    meta = []
    for tag in target_tds:
        label = tag.text.strip(':')
        content = tag.find_next('td', {'class' : 'tableText'}).text
        meta.append({'label' : label,
                     'content' : content})
    for tag in target_ths:
        label = tag.text.strip(':')
        content = tag.find_next('td', {'class' : 'tableText'})
        if label == 'Sponsors':
            values = content.find_all('a')
            sponsors = []
            for v in values:
                sponsors.append({'name' : v.text,
                                 'link' : v['href']})
            content = sponsors
        else:
            content = content.text
        meta.append({'label' : label,
                     'content' : content})
    
    return(meta)

In [108]:
pdf['meta'] = pdf.soup.apply(get_meta)

In [109]:
pdf['meta'][3]

[{'content': 'Re-ref Com On Finance on 03/08/2017', 'label': 'Last Action'},
 {'content': 'Public; Text has changed;       ', 'label': 'Attributes'},
 {'content': [{'link': '/gascripts/members/viewMember.pl?sChamber=H&nUserID=489',
    'name': 'Dollar'},
   {'link': '/gascripts/members/viewMember.pl?sChamber=H&nUserID=632',
    'name': 'Saine'},
   {'link': '/gascripts/members/viewMember.pl?sChamber=H&nUserID=645',
    'name': 'Hardister'},
   {'link': '/gascripts/members/viewMember.pl?sChamber=H&nUserID=658',
    'name': 'R.\xa0Turner'},
   {'link': '/gascripts/members/viewMember.pl?sChamber=H&nUserID=688',
    'name': 'Adcock'},
   {'link': '/gascripts/members/viewMember.pl?sChamber=H&nUserID=661',
    'name': 'J.\xa0Bell'},
   {'link': '/gascripts/members/viewMember.pl?sChamber=H&nUserID=690',
    'name': 'Bradford'},
   {'link': '/gascripts/members/viewMember.pl?sChamber=H&nUserID=691',
    'name': 'Brockman'},
   {'link': '/gascripts/members/viewMember.pl?sChamber=H&nUserID=582',


## Session & Bill Editions

In [111]:
session = soup.find('div', {'class' : 'titleSub'})
session.text

'2017-2018 Session'

In [114]:
editions_table = session.next_sibling.find('table')

In [127]:
entries = editions_table.find_all('tr')

In [128]:
len(entries)

5

In [ ]:
bill_summaries = entries[0]

In [132]:
header = entries[1]
header_values = [th.text for th in header.find_all('th')]
header_values

['Bill Text', 'Fiscal Note']

In [133]:
def get_edition_content(target_rows):

    edition_content = []

    for entry in target_rows:
        tmp = []
        cols = entry.find_all('td')
        for c,h in zip(cols, header_values):
            if h == 'Bill Text':
                links = {'pdf' : '', 'html' : ''}
                vals = c.find_all('a')
                for v in vals:
                    t = v.text.strip()
                    link = v['href']
                    if t.lower() == 'html':
                        links['html'] = link
                    else:
                        edition = t
                        links['pdf'] = link
                tmp.append({'Links' : links})
            elif h == 'Fiscal Note':
                tmp.append({h : c.text.strip()})
        edition_content.append({edition : tmp})
        
    return(edition_content)

In [134]:
edition_content

[{'Filed': [{'Links': {'html': '/Sessions/2017/Bills/House/HTML/H1v0.html',
     'pdf': '/Sessions/2017/Bills/House/PDF/H1v0.pdf'}},
   {'Fiscal Note': ''}]},
 {'Edition\xa01': [{'Links': {'html': '/Sessions/2017/Bills/House/HTML/H1v1.html',
     'pdf': '/Sessions/2017/Bills/House/PDF/H1v1.pdf'}},
   {'Fiscal Note': ''}]},
 {'Adopted': [{'Links': {'html': '/Sessions/2017/Bills/House/HTML/H1v2.html',
     'pdf': '/Sessions/2017/Bills/House/PDF/H1v2.pdf'}},
   {'Fiscal Note': ''}]}]

In [140]:
def get_session_and_editions(soup):
    
    def get_edition_content(target_rows, header_values):

        edition_content = []

        for entry in target_rows:
            tmp = []
            cols = entry.find_all('td')
            for c,h in zip(cols, header_values):
                if h == 'Bill Text':
                    links = {'pdf' : '', 'html' : ''}
                    vals = c.find_all('a')
                    for v in vals:
                        t = v.text.strip()
                        link = v['href']
                        if t.lower() == 'html':
                            links['html'] = link
                        else:
                            edition = t
                            links['pdf'] = link
                    tmp.append({'Links' : links})
                elif h == 'Fiscal Note':
                    tmp.append({h : c.text.strip()})
            edition_content.append({edition : tmp})

        return(edition_content)


    content = []
    
    # Session Info
    session = soup.find('div', {'class' : 'titleSub'})
    content.append({'label' : 'Session',
                    'content' : session.text.strip()})
    
    # Bill Editions, Summary
    editions_table = session.next_sibling.find('table')
    entries = editions_table.find_all('tr')
    
    bill_summaries = entries[0]
    content.append({'label' : 'SummaryLink',
                    'content' : bill_summaries.find('a')['href']})
    
    header = entries[1]
    header_values = [th.text for th in header.find_all('th')]    
    edition_content = get_edition_content(entries[2:], header_values)
    content.append({'label' : 'EditionsInfo',
                    'content' : edition_content})
    
    return(content)

In [143]:
for v in get_session_and_editions(soup):
    print(v['label'])
    print('\t' + str(v['content']))

Session
	2017-2018 Session
SummaryLink
	/gascripts/billsummaries/billsummaries.pl?Session=2017&BillID=H1
EditionsInfo
	[{'Filed': [{'Links': {'html': '/Sessions/2017/Bills/House/HTML/H1v0.html', 'pdf': '/Sessions/2017/Bills/House/PDF/H1v0.pdf'}}, {'Fiscal Note': ''}]}, {'Edition\xa01': [{'Links': {'html': '/Sessions/2017/Bills/House/HTML/H1v1.html', 'pdf': '/Sessions/2017/Bills/House/PDF/H1v1.pdf'}}, {'Fiscal Note': ''}]}, {'Adopted': [{'Links': {'html': '/Sessions/2017/Bills/House/HTML/H1v2.html', 'pdf': '/Sessions/2017/Bills/House/PDF/H1v2.pdf'}}, {'Fiscal Note': ''}]}]


In [144]:
pdf['editions'] = pdf.soup.apply(get_session_and_editions)

In [145]:
pdf['editions'][3]

[{'content': '2017-2018 Session', 'label': 'Session'},
 {'content': '/gascripts/billsummaries/billsummaries.pl?Session=2017&BillID=H1',
  'label': 'SummaryLink'},
 {'content': [{'Filed': [{'Links': {'html': '/Sessions/2017/Bills/House/HTML/H1v0.html',
       'pdf': '/Sessions/2017/Bills/House/PDF/H1v0.pdf'}},
     {'Fiscal Note': ''}]},
   {'Edition\xa01': [{'Links': {'html': '/Sessions/2017/Bills/House/HTML/H1v1.html',
       'pdf': '/Sessions/2017/Bills/House/PDF/H1v1.pdf'}},
     {'Fiscal Note': ''}]},
   {'Adopted': [{'Links': {'html': '/Sessions/2017/Bills/House/HTML/H1v2.html',
       'pdf': '/Sessions/2017/Bills/House/PDF/H1v2.pdf'}},
     {'Fiscal Note': ''}]}],
  'label': 'EditionsInfo'}]